# Traitement des valeurs manquantes



Points traités dans ce notebook :

- une 1ère aperçu des données
- Voir combien de points de données manquants nous avons
- Déterminer la raison pour laquelle les données sont manquantes
- Supprimer les valeurs manquantes
- Compléter les valeurs manquantes

# 1ère aperçu des données


j'exploite un ensemble de données sur les événements survenus lors des matchs de football américain à des fins de démonstration, et un autre df sur les permis de construire délivrés à San Francisco.

In [2]:
# modules que nous utiliserons
import pandas as pd
import numpy as np

# lire toutes nos données
nfl_data = pd.read_csv("/Users/jl/Downloads/NFL Play by Play 2009-2017 (v4).csv")
sf_permits = pd.read_csv("/Users/jl/Downloads/Building_Permits.csv")

# reproductibilité
np.random.seed(0) 

/var/folders/r5/sycktgpj0ln8vbwphhbh5mb00000gn/T/ipykernel_14860/788546147.py:6: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  nfl_data = pd.read_csv("/Users/jl/Downloads/NFL Play by Play 2009-2017 (v4).csv")
/var/folders/r5/sycktgpj0ln8vbwphhbh5mb00000gn/T/ipykernel_14860/788546147.py:7: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  sf_permits = pd.read_csv("/Users/jl/Downloads/Building_Permits.csv")


La première chose que je fais lorsque je reçois un nouvel ensemble de données est d'en examiner une partie.
Cela me permet de vérifier que tout a été lu correctement et d'avoir une idée de ce qui se passe avec les données.
Dans ce cas, je cherche à voir s'il y a des valeurs manquantes, qui seront représentées par NaN ou None.

In [3]:
# Je regarde quelques lignes du fichier nfl_data. Je vois déjà une poignée de données manquantes
nfl_data.sample(5)

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
244485,2014-10-26,2014102607,18,3,1.0,00:39,1,939.0,12.0,TB,...,1.240299,0.225647,0.774353,0.245582,0.754418,0.225647,0.019935,-0.018156,0.038091,2014
115340,2011-11-20,2011112000,22,4,1.0,06:47,7,407.0,44.0,OAK,...,NaN,0.056036,0.943964,0.042963,0.957037,0.943964,0.013073,NaN,NaN,2011
68357,2010-11-14,2010111401,8,2,NaN,00:23,1,1823.0,0.0,CLE,...,NaN,0.365307,0.634693,0.384697,0.615303,0.634693,-0.019390,NaN,NaN,2010
368377,2017-09-24,2017092405,24,4,1.0,08:48,9,528.0,8.0,CLE,...,1.075660,0.935995,0.064005,0.921231,0.078769,0.064005,0.014764,0.003866,0.010899,2017
384684,2017-11-05,2017110505,11,2,1.0,09:15,10,2355.0,0.0,DEN,...,NaN,0.928474,0.071526,0.934641,0.065359,0.071526,-0.006166,NaN,NaN,2017


il y a bien des valeurs manquantes. Qu'en est-il de l'ensemble de données sf_permits ?

In [5]:
# Je regarde quelques lignes du fichier nfl_data. Je vois déjà une poignée de données manquantes
sf_permits.sample(5)
# idem

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
49009,201405226535,8,otc alterations permit,05/22/2014,0997,017A,200,NaN,Walnut,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,2.0,Presidio Heights,94118.0,"(37.78958158007647, -122.44873860281018)",134291486469
46655,201405014687,8,otc alterations permit,05/01/2014,1219,029,1263,NaN,Oak,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,NaN,NaN,NaN,NaN,1340469240465
74211,201501297005,8,otc alterations permit,01/29/2015,0334,004,325,NaN,Leavenworth,St,...,3.0,constr type 3,3.0,constr type 3,NaN,6.0,Tenderloin,94109.0,"(37.78405819312474, -122.41456846296983)",136938469855
55407,201407221800,8,otc alterations permit,07/22/2014,0857,001,218,NaN,Buchanan,St,...,NaN,NaN,NaN,NaN,NaN,8.0,Hayes Valley,94102.0,"(37.77234985626402, -122.42627871236607)",1349630492927
17030,201306270613,8,otc alterations permit,06/27/2013,5891,015,119,NaN,Maynard,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,11.0,Excelsior,94112.0,"(37.72952811045384, -122.42781140712628)",1309434190479


# Voir combien de points de données manquants nous avons

nous avons quelques valeurs manquantes. Voyons combien nous en avons dans chaque colonne.

In [11]:
# déterminer le nombre de points de données manquants par colonne
missing_values_count = nfl_data.isnull().sum()


In [13]:
# visualiser le nombre de points manquants dans les dix premières colonnes
missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

Cela semble beaucoup ! Il serait peut-être utile de connaître le pourcentage de valeurs manquantes dans notre ensemble de données pour nous donner une meilleure idée de l'ampleur du problème :

In [16]:
# Combien de valeurs manquantes avons-nous au total ?
total_cells = np.product(nfl_data.shape)
total_missing = missing_values_count.sum()

# Pourcentage de données manquantes
(total_missing/total_cells) * 100

24.87214126835169

Près d'un quart des cellules de cet ensemble de données sont vides !

Dans l'étape suivante, nous allons examiner de plus près certaines des colonnes présentant des valeurs manquantes et essayer de comprendre ce qui se passe.

In [24]:
# quel pourcentage de l'ensemble de données sf_permits est manquant

# déterminer le nombre de points de données manquants par colonne
missing_values_count_sf_permits = sf_permits.isnull().sum()

# visualiser le nombre de points manquants dans les dix premières colonnes
missing_values_count_sf_permits[0:10]




Permit Number                  0
Permit Type                    0
Permit Type Definition         0
Permit Creation Date           0
Block                          0
Lot                            0
Street Number                  0
Street Number Suffix      196684
Street Name                    0
Street Suffix               2768
dtype: int64

In [28]:
# Combien de valeurs manquantes avons-nous au total ?
total_cells_sf_permits = np.product(sf_permits.shape)

total_missing_sf_permits = missing_values_count_sf_permits.sum()

# Pourcentage de données manquantes
(total_missing_sf_permits/total_cells_sf_permits) * 100


26.26002315058403

# Déterminer pourquoi les données sont manquantes

Pour traiter les valeurs manquantes, il faut faire preuve d'intuition pour comprendre pourquoi la valeur est manquante.

Question à se poser pour résoudre ce problème :

Cette valeur est-elle manquante parce qu'elle n'a pas été enregistrée ou parce qu'elle n'existe pas ?

Si une valeur est manquante parce qu'elle n'existe pas (comme la taille de l'enfant le plus âgé d'une personne qui n'a pas d'enfant), il est inutile d'essayer de deviner ce qu'elle pourrait être. Il est probable que je souhaite conserver ces valeurs sous forme de NaN, dans ce cas.

En revanche, si une valeur est manquante parce qu'elle n'a pas été enregistrée, je vais essayer de deviner ce qu'elle aurait pu être en me basant sur les autres valeurs de cette colonne et de cette ligne. C'est ce qu'on appelle l' "imputation". Je vais aborder cette notion par la suite.



In [30]:
# Prenons un exemple. En regardant le nombre de valeurs manquantes dans le df nfl_data, je remarque que la colonne TimesSec contient beaucoup de valeurs manquantes :

# regarder le nombre de points manquants dans les dix premières colonnes
missing_values_count[0:10]



Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

En consultant la documentation, je constate que cette colonne contient des informations sur le nombre de secondes restantes dans le jeu lorsque le jeu a été effectué. Cela signifie que ces valeurs sont probablement manquantes parce qu'elles n'ont pas été enregistrées, plutôt que parce qu'elles n'existent pas. Il serait donc logique que j'essaie de deviner ce qu'elles devraient être plutôt que de les laisser comme NA.

D'autre part, d'autres champs, comme PenalizedTeam, comportent également de nombreux champs manquants. Dans ce cas, cependant, le champ est manquant parce que s'il n'y a pas eu de pénalité, cela n'a pas de sens de dire quelle équipe a été pénalisée. Pour cette colonne, il serait plus logique de la laisser vide ou d'ajouter une troisième valeur comme "neither" et de l'utiliser pour remplacer les NA.

Bonne pratique : si l'on travaille avec un jeu de données que nous avons obtenu d'une autre personne, on peut également essayer de la contacter pour obtenir plus d'informations.

Si j'effectue une analyse très minutieuse des données, c'est à ce moment-là que j'examinerai chaque colonne individuellement afin de déterminer la meilleure stratégie pour combler les valeurs manquantes.
Dans la suite de ce notebook, j'aborderai quelques techniques "rapides et sales" qui m'aideront à combler les valeurs manquantes, mais qui finiront probablement par supprimer des informations utiles ou par ajouter du bruit à mes données.



In [37]:
# Examinons les colonnes Street Number Suffix et Zipcode de l'ensemble de données sf_permits
sf_permits.sample(60)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
41194,201403070180,8,otc alterations permit,03/07/2014,0322,001,601,NaN,Ofarrell,St,...,3.0,constr type 3,3.0,constr type 3,NaN,6.0,Tenderloin,94109.0,"(37.78528309870539, -122.41478641316387)",133475969504
144680,201612094578,6,demolitions,12/09/2016,0859,022,228,NaN,Fillmore,St,...,5.0,wood frame (5),NaN,NaN,NaN,5.0,Hayes Valley,94117.0,"(37.77163988547394, -122.43008085845331)",144691483298
27305,201310088810,3,additions alterations or repairs,10/08/2013,3564,070A,355,NaN,Noe,St,...,5.0,wood frame (5),5.0,wood frame (5),Y,8.0,Castro/Upper Market,94114.0,"(37.76334686651608, -122.4327133688931)",1320215435912
123178,201605056661,3,additions alterations or repairs,05/05/2016,1459,011,478,NaN,28th,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,1.0,Outer Richmond,94121.0,"(37.780442718742684, -122.48767416153478)",1422154100611
85836,M587687,8,otc alterations permit,05/15/2015,2752,026,321,NaN,Collingwood,St,...,NaN,NaN,NaN,NaN,NaN,8.0,Castro/Upper Market,94114.0,"(37.75721787999552, -122.43545075658255)",1381792418685
89363,201506189294,8,otc alterations permit,06/18/2015,0306,006,432,NaN,Geary,St,...,3.0,constr type 3,3.0,constr type 3,NaN,3.0,Tenderloin,94102.0,"(37.78726475349522, -122.41037579292663)",138553367356
9777,M385908,8,otc alterations permit,04/16/2013,1742,034,1220,NaN,09th,Av,...,NaN,NaN,NaN,NaN,NaN,5.0,Inner Sunset,94122.0,"(37.765501505876266, -122.46608376737706)",1301844363618
115559,201602240395,8,otc alterations permit,02/24/2016,3569,014,2072,NaN,Mission,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,9.0,Mission,94110.0,"(37.76384499486115, -122.41987268743213)",1413789404944
14974,201306078935,8,otc alterations permit,06/07/2013,1647,036,700,NaN,05th,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,1.0,Inner Richmond,94118.0,"(37.775286352650575, -122.46251703251735)",1307285108390
21460,201308134178,8,otc alterations permit,08/13/2013,1135,025,821,NaN,Anza,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,1.0,Lone Mountain/USF,94118.0,"(37.779881880252844, -122.45513719478035)",1314049492418


En regardant les colonnes Street Number Suffix et Zipcode de l'ensemble de données sf_permits, ces deux colonnes contiennent des valeurs manquantes.

Les valeurs de la colonne Street Number Suffix sont normalement manquantes parce qu'elles n'existent pas

Certaines valeurs de la colonne Zipcode sont manquantes parce qu'elles n'ont pas été enregistrées

# Supprimer les valeurs manquantes

Si l'on est pris par le temps, on peut supprimer les lignes ou les colonnes qui contiennent des valeurs manquantes.
Remarque : je ne recommande généralement pas cette approche pour les projets importants ! Il vaut généralement la peine de prendre le temps de parcourir les données et d'examiner attentivement toutes les colonnes contenant des valeurs manquantes, une par une, afin d'apprendre à connaître notre ensemble de données.

Si l'on veut supprimer les lignes avec des valeurs manquantes, pandas dispose d'une fonction pratique, dropna(), pour nous aider à le faire. Je vais la tester sur notre jeu de données NFL

In [38]:
# supprimer toutes les lignes contenant une valeur manquante
nfl_data.dropna()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


Toutes nos données ont été supprimées !
En effet, chaque ligne de notre ensemble de données comporte au moins une valeur manquante. Nous pourrions avoir plus de chance en supprimant toutes les colonnes qui ont au moins une valeur manquante à la place.

In [39]:
# supprimer toutes les colonnes avec au moins une valeur manquante
columns_with_na_dropped = nfl_data.dropna(axis=1)
columns_with_na_dropped.head()

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,...,Timeout_Indicator,Timeout_Team,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,0,None,3,3,3,3,3,0.0,0.0,2009
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,0,None,3,3,3,3,3,0.0,0.0,2009


In [40]:
# combien de données avons-nous perdues ?
print("Columns in original dataset: %d \n" % nfl_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 102 

Columns with na's dropped: 41


Nous avons perdu pas mal de données, mais à ce stade, nous avons réussi à supprimer tous les NaN de nos données.


In [43]:
# Nous allons supprimer toutes les lignes de l'ensemble de données sf_permits qui contiennent des valeurs manquantes

sf_permits.dropna()

# Combien en reste-t-il ? = 0 ligne

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID


In [44]:
# supprimer toutes les colonnes contenant des valeurs vides (avec au moins une valeur manquante)

columns_with_na_dropped_sf_permits = sf_permits.dropna(axis=1)
columns_with_na_dropped_sf_permits.head()


# Quelle quantité de données reste-t-il ? 5 lignes

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Current Status,Current Status Date,Filed Date,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,Ellis,expired,12/21/2017,05/06/2015,1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,Geary,issued,08/03/2017,04/19/2016,1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,Pacific,withdrawn,09/26/2017,05/27/2016,1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,Pacific,complete,07/24/2017,11/07/2016,1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,Market,issued,12/01/2017,11/28/2016,144548169992


In [45]:
# combien de données avons-nous perdues ?
print("Columns in original dataset: %d \n" % sf_permits.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped_sf_permits.shape[1])

Columns in original dataset: 43 

Columns with na's dropped: 12


# Remplir les valeurs manquantes automatiquement ( = imputation)

Une autre option consiste à essayer de remplir les valeurs manquantes. Pour ce qui suit, j'obtiendrai une petite partie des données de la NFL afin que l'impression se passe bien.

In [46]:
# obtenir un petit sous-ensemble de l'ensemble de données NFL
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
subset_nfl_data

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,NaN,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,NaN,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,NaN,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


Nous pouvons utiliser la fonction fillna() de Pandas pour remplir les valeurs manquantes d'une base de données. L'une des options dont nous disposons est de spécifier par quoi nous voulons que les valeurs NaN soient remplacées. Ici, je dis que j'aimerais remplacer toutes les valeurs NaN par 0.

In [47]:
# remplacer tous les NA par 0
subset_nfl_data.fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


Je pourrais également être un peu plus avisé et remplacer les valeurs manquantes par la valeur qui les suit directement dans la même colonne.
Cela a beaucoup de sens pour les ensembles de données où les observations ont une sorte d'ordre logique.

In [52]:
# remplacer tous les NA par la valeur qui vient directement après dans la même colonne, 
# puis remplacer tous les NA qui suivent par 0
subset_nfl_data.fillna(method = 'bfill', axis=0).fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,-1.068169,1.146076,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,-0.032244,0.036899,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,3.318841,-5.031425,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.106663,-0.156239,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009


In [50]:
# Rappel : Remplir les valeurs manquantes est également connu sous le nom d'"imputation"